In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import os
import re
import math

# コンテストの概要
## 与えられるもの  
人工呼吸器の時系列データ  
・inspratory(吸った息)側呼吸器回路のバルブの開閉度(0~100)  
・expiratory(吐いた息)側呼吸器回路の弁(0 or 1)  
・肺の属性値(time_stepによらず一定、breath_idにより値が変化(それぞれ３種類ずつ))  
　・R(抵抗)　… 気道がどの程度制限されているかを表す値、大きいほど抵抗が強い  
　・C(コンプライアンス)　… 肺のコンプライアンスを表す値、風船をイメージすると膜厚を変更することによってコンプライアンスを調整可能。Cが大きくなるほど風船は吹きやすい。  
・breath_id … 各試行につけられた番号、試行数は125,750個存在し、そのうちの7.5万個が訓練用、5万個がテスト用として使用  
## 予測するもの
・Pressure … expiratory側で測定された空気の圧力  
※cmH2Oは水中センチメートルと読む圧力の単位

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/sample_submission.csv')
train_data = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/train.csv')
test_data = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/test.csv')

In [ ]:
train_data.head()

In [ ]:
test_data.head()

# EDA

In [ ]:
print('train_dataの要素数', train_data.shape)
print('test_dataの要素数', test_data.shape)

In [ ]:
print('train_dataのユニークなリスト', train_data['breath_id'].unique())
print('train_dataのユニークな値の個数', train_data['breath_id'].nunique())
print('test_dataのユニークなリスト', test_data['breath_id'].unique())
print('test_dataのユニークな値の個数', test_data['breath_id'].nunique())

In [ ]:
groupby_breath_id = train_data.groupby('breath_id').mean()
print('Rのユニークな値', groupby_breath_id['R'].unique())
print('Cのユニークな値' ,groupby_breath_id['C'].unique())

In [ ]:
groupby_breath_id.describe()

In [ ]:
train_3 = train_data.loc[train_data['breath_id'] == 1].head()

In [ ]:
train_0 = train_data.loc[train_data['breath_id'] == 1000].head()
train_0.head()

In [ ]:
plt.figure(figsize=(16,9))
plt.plot(train_3['time_step'], train_3['pressure'], label='pressure')
plt.plot(train_3['time_step'], train_3['u_in'], label='u_in')
plt.plot(train_3['time_step'], train_3['u_out'], label='u_out')
plt.legend()
plt.show()